<a href="https://colab.research.google.com/github/eriksali/DNN_2023_NLP/blob/main/NLP_logistic_regression_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
CSI 5900: Lecture 11 Code Examples
Prof. Steven Wilson, Oakland University

Logistic Regression for Binary Classification

We will use a dataset of IMDB reviews from:

Andrew L. Maas, Raymond E. Daly, Peter T. Pham, Dan Huang, Andrew Y. Ng, and Christopher Potts. (2011). 
"Learning Word Vectors for Sentiment Analysis." The 49th Annual Meeting of the Association for Computational Linguistics (ACL 2011).
'''

! wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
! tar -xzf aclImdb_v1.tar.gz

! cat aclImdb/README

import glob
pos_train_files = glob.glob('aclImdb/train/pos/*')
neg_train_files = glob.glob('aclImdb/train/neg/*')
print(pos_train_files[:5]) 

from sklearn.feature_extraction.text import TfidfVectorizer
# only use 1000 data points per class for now to make things faster/simpler
num_files_per_class = 1000
all_train_files = pos_train_files[:num_files_per_class] + neg_train_files[:num_files_per_class]
vectorizer = TfidfVectorizer(input="filename", stop_words="english")
vectors = vectorizer.fit_transform(all_train_files)
vectors

len(vectorizer.vocabulary_)

vectors[0].sum()

X = vectors
y = [1] * num_files_per_class + [0] * num_files_per_class
len(y)

import numpy as np
x_0 = X[0]
w = np.zeros(X.shape[1])
x_0_dense = x_0.todense()
x_0.dot(w)

import random
import numpy as np
from scipy.special import expit

def sgd_for_lr_with_ce(X, y, num_passes=5, learning_rate = 0.1):

    num_data_points = X.shape[0]

    # Initialize theta -> 0
    num_features = X.shape[1]
    w = np.zeros(num_features)
    b = 0.0

    # repeat until done
    # how to define "done"? let's just make it num passes for now
    # we can also do norm of gradient and when it is < epsilon (something tiny)
    # we stop

    for current_pass in range(num_passes):
        
        # iterate through entire dataset in random order
        order = list(range(num_data_points))
        random.shuffle(order)
        for i in order:

            # compute y-hat for this value of i given y_i and x_i
            x_i = X[i]
            y_i = y[i]

            # need to compute based on w and b
            # sigmoid(w dot x + b)
            z = x_i.dot(w) + b
            y_hat_i = expit(z)

            # for each w (and b), modify by -lr * (y_hat_i - y_i) * x_i
            w = w - learning_rate * (y_hat_i - y_i) * x_i
            b = b - learning_rate * (y_hat_i - y_i)

    # return theta
    return w,b

w,b = sgd_for_lr_with_ce(X,y)

w

sorted_vocab = sorted([(k,v) for k,v in vectorizer.vocabulary_.items()],key=lambda x:x[1])
sorted_vocab = [a for (a,b) in sorted_vocab]

sorted_words_weights = sorted([x for x in zip(sorted_vocab, w)], key=lambda x:x[1])
sorted_words_weights[-50:]

# get the predictions
def predict_y_lr(w,b,X,threshold=0.5):

    # use our matrix operation version of the logistic regression model
    # X dot w + b
    # need to make w a column vector so the dimensions line up correctly
    y_hat = X.dot( w.reshape((-1,1)) ) + b

    # then just check if it's > threshold
    preds = np.where(y_hat > threshold,1,0)

    return preds

preds = predict_y_lr(w,b,X)

preds

# compute training set results
from sklearn.metrics import classification_report
w,b = sgd_for_lr_with_ce(X, y, num_passes=10)
y_pred = predict_y_lr(w,b,X)
print(classification_report(y, y_pred))

# compute for dev set
pos_dev_files = glob.glob('aclImdb/test/pos/*')
neg_dev_files = glob.glob('aclImdb/test/neg/*')
num_dev_files_per_class = 100
all_dev_files = pos_dev_files[:num_dev_files_per_class] + neg_dev_files[:num_dev_files_per_class]
# use the same vectorizer from before! otherwise features won't line up
# don't fit it again, just use it to transform!
X_dev = vectorizer.transform(all_dev_files)
y_dev = [1]* num_dev_files_per_class + [0]* num_dev_files_per_class
# don't need new w and b, these are from out existing model
y_dev_pred = predict_y_lr(w,b,X_dev)
print(classification_report(y_dev, y_dev_pred))



--2023-03-16 19:39:54--  https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  25.6MB/s    in 3.1s    

2023-03-16 19:39:57 (25.6 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]

Large Movie Review Dataset v1.0

Overview

This dataset contains movie reviews along with their associated binary
sentiment polarity labels. It is intended to serve as a benchmark for
sentiment classification. This document outlines how the dataset was
gathered, and how to use the files provided. 

Dataset 

The core dataset contains 50,000 reviews split evenly into 25k train
and 25k test sets. The overall distribution of labels is balanced (25k
pos and 25k neg). We also include an